In [2]:
import datetime 

current_date= datetime.datetime.now().strftime('%Y-%m-%d')
print("print current date :",current_date)

seven_days_ago = (datetime.datetime.now() - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
print("print 7 days ago ",seven_days_ago)

print current date : 2025-03-28
print 7 days ago  2025-03-21


In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

news_api=os.getenv("NEWS_API")

In [4]:
from newsapi import NewsApiClient 

newsapi = NewsApiClient(news_api)

In [5]:
all_articles_gen_Ai = newsapi.get_everything(q='GenAI',
                                      from_param=seven_days_ago,
                                      to=current_date,
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)



In [6]:
all_articles_gen_Ai

{'status': 'ok',
 'totalResults': 252,
 'articles': [{'source': {'id': None, 'name': 'Pypi.org'},
   'author': 'ragbits@deepsense.ai',
   'title': 'ragbits 0.12.0',
   'description': 'Building blocks for rapid development of GenAI applications',
   'url': 'https://pypi.org/project/ragbits/0.12.0/',
   'urlToImage': None,
   'publishedAt': '2025-03-25T17:53:18Z',
   'content': 'A required part of this site couldnt load. This may be due to a browser\r\n extension, network issues, or browser settings. Please check your\r\n connection, disable any ad blockers, or try using a diffe… [+12 chars]'},
  {'source': {'id': None, 'name': 'Pypi.org'},
   'author': 'ragbits@deepsense.ai',
   'title': 'ragbits 0.10.2',
   'description': 'Building blocks for rapid development of GenAI applications',
   'url': 'https://pypi.org/project/ragbits/0.10.2/',
   'urlToImage': None,
   'publishedAt': '2025-03-21T15:24:39Z',
   'content': 'A required part of this site couldnt load. This may be due to a browser

In [7]:
all_articles_ai = newsapi.get_everything(q='AI',
                                      from_param=seven_days_ago,
                                      to=current_date,
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

In [8]:
all_articles_ai

{'status': 'ok',
 'totalResults': 9518,
 'articles': [{'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Joe Tilleli',
   'title': 'Roborock S8 MaxV Ultra: The Best Robot Vacuum and Mop Is Now 45% Off, Amazon Is Going Wild',
   'description': 'Let this robot vacuum and mop combo take care of the tedious chore of keeping your floors clean.',
   'url': 'https://gizmodo.com/roborock-s8-maxv-ultra-the-best-robot-vacuum-and-mop-is-now-45-off-amazon-is-going-wild-2000580720',
   'urlToImage': 'https://gizmodo.com/app/uploads/2025/03/roborockS8.jpg',
   'publishedAt': '2025-03-26T12:10:26Z',
   'content': 'AI has become quite a contentious topic these past couple years. It seems everyone is rushing at using services like ChatGPT or Midjourney to write scripts or put together videos to watch. These folk… [+2269 chars]'},
  {'source': {'id': None, 'name': 'Android Central'},
   'author': 'michael.hicks@futurenet.com (Michael L Hicks)',
   'title': 'Garmin Connect+ subscription adds A

In [9]:
all_articles_technology = newsapi.get_everything(q='Technology',
                                      from_param=seven_days_ago,
                                      to=current_date,
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

In [10]:
all_articles_technology

{'status': 'ok',
 'totalResults': 11250,
 'articles': [{'source': {'id': None, 'name': 'PetaPixel'},
   'author': 'Pesala Bandara',
   'title': 'AI-Generated Food Images Can Make People Feel Uncomfortable',
   'description': 'New research has revealed why AI-generated food images can make people uneasy -- raising questions about the potential of the technology to replace traditional food photography in ads.\n[Read More]',
   'url': 'https://petapixel.com/2025/03/27/ai-generated-food-images-can-make-people-feel-uncomfortable/',
   'urlToImage': 'https://petapixel.com/assets/uploads/2025/03/cheeseburger-ai-generated.jpg',
   'publishedAt': '2025-03-27T12:32:19Z',
   'content': 'An AI-generated image of a cheeseburger. Created with DALL·E\r\nNew research has revealed why AI-generated food images can make people uneasy — raising questions about the potential of the technology t… [+2224 chars]'},
  {'source': {'id': 'business-insider', 'name': 'Business Insider'},
   'author': 'Mia Jankowic

In [11]:
combined_articles=all_articles_ai['articles'] + all_articles_gen_Ai['articles'] + all_articles_technology['articles']

In [12]:
import pandas as pd
combined_articles_df=pd.DataFrame(combined_articles)
combined_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Gizmodo.com'}",Joe Tilleli,Roborock S8 MaxV Ultra: The Best Robot Vacuum ...,Let this robot vacuum and mop combo take care ...,https://gizmodo.com/roborock-s8-maxv-ultra-the...,https://gizmodo.com/app/uploads/2025/03/roboro...,2025-03-26T12:10:26Z,AI has become quite a contentious topic these ...
1,"{'id': None, 'name': 'Android Central'}",michael.hicks@futurenet.com (Michael L Hicks),Garmin Connect+ subscription adds Active Intel...,"""All existing features and data in Garmin Conn...",https://www.androidcentral.com/wearables/new-g...,https://cdn.mos.cms.futurecdn.net/XSc3G7YTxWhd...,2025-03-27T11:00:00Z,What you need to know\r\n<ul><li>Garmin Connec...
2,"{'id': None, 'name': 'Android Central'}",nandika.iyerravi@futurenet.com (Nandika Ravi),Gemini could soon allow you to upload multiple...,Gemini could allow users to start uploading up...,https://www.androidcentral.com/apps-software/g...,https://cdn.mos.cms.futurecdn.net/YSVyEAy8kCuA...,2025-03-25T19:55:14Z,What you need to know\r\n<ul><li>An APK teardo...
3,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Apple Still Planning AirPods With Tiny Cameras,Apple continues to explore the idea of adding ...,https://www.macrumors.com/2025/03/22/apple-sti...,https://images.macrumors.com/t/hZNqVW5CIFNjQp6...,2025-03-22T15:20:00Z,Apple continues to explore the idea of adding ...
4,"{'id': None, 'name': 'Android Central'}",nickodiaz@sbcglobal.net (Nickolas Diaz),Samsung's sixth One UI 7 beta for the Galaxy S...,Samsung reportedly started rolling out its six...,https://www.androidcentral.com/phones/samsung-...,https://cdn.mos.cms.futurecdn.net/jpswqvttvAMa...,2025-03-27T17:53:18Z,What you need to know\r\n<ul><li>Samsung is re...
...,...,...,...,...,...,...,...,...
295,"{'id': None, 'name': 'Core77.com'}",Rain Noe,Fantastic Industrial Design: The Kodak Smile C...,When Kodak wanted to produce an instant digita...,https://www.core77.com/posts/136031/Fantastic-...,https://s3files.core77.com/blog/images/lead_n_...,2025-03-24T16:00:00Z,When Kodak wanted to produce an instant digita...
296,"{'id': 'business-insider', 'name': 'Business I...",jmcgrath@insider.com (Jenny McGrath),A sheet's thread count isn't as important as y...,"When it comes to sheets, you'll want to consid...",https://www.businessinsider.com/guides/home/wh...,https://i.insider.com/62a209202253ea001951eb65...,2025-03-24T19:27:52Z,Thread count referring to the number of yarns ...
297,"{'id': None, 'name': 'Lifeinsuranceinternation...",GlobalData,1st Central adopts Clearspeed voice-based tech...,This move aims to expedite the settlement of l...,https://www.lifeinsuranceinternational.com/new...,https://s.yimg.com/ny/api/res/1.2/.Lmcn2EcLIqy...,2025-03-25T16:45:07Z,"1st Central, a UK-based insurer, has integrate..."
298,"{'id': None, 'name': 'Substack.com'}",UNTOPIA,The Tranhumanist Cult Test,Transhumanism is put through the 8-point test ...,https://ewanmorrison.substack.com/p/the-tranhu...,"https://substackcdn.com/image/fetch/w_1200,h_6...",2025-03-24T12:06:16Z,In trying to have a few conversations about tr...


In [22]:
combined_articles_df.drop('urlToImage',axis=1)

,source,author,title,description,url,publishedAt,content
0,"{'id': None, 'name': 'Gizmodo.com'}",Joe Tilleli,Roborock S8 MaxV Ultra: The Best Robot Vacuum ...,Let this robot vacuum and mop combo take care ...,https://gizmodo.com/roborock-s8-maxv-ultra-the...,2025-03-26T12:10:26Z,AI has become quite a contentious topic these ...
1,"{'id': None, 'name': 'Android Central'}",michael.hicks@futurenet.com (Michael L Hicks),Garmin Connect+ subscription adds Active Intel...,"""All existing features and data in Garmin Conn...",https://www.androidcentral.com/wearables/new-g...,2025-03-27T11:00:00Z,What you need to know\r\n<ul><li>Garmin Connec...
2,"{'id': None, 'name': 'Android Central'}",nandika.iyerravi@futurenet.com (Nandika Ravi),Gemini could soon allow you to upload multiple...,Gemini could allow users to start uploading up...,https://www.androidcentral.com/apps-software/g...,2025-03-25T19:55:14Z,What you need to know\r\n<ul><li>An APK teardo...
3,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Apple Still Planning AirPods With Tiny Cameras,Apple continues to explore the idea of adding ...,https://www.macrumors.com/2025/03/22/apple-sti...,2025-03-22T15:20:00Z,Apple continues to explore the idea of adding ...
4,"{'id': None, 'name': 'Android Central'}",nickodiaz@sbcglobal.net (Nickolas Diaz),Samsung's sixth One UI 7 beta for the Galaxy S...,Samsung reportedly started rolling out its six...,https://www.androidcentral.com/phones/samsung-...,2025-03-27T17:53:18Z,What you need to know\r\n<ul><li>Samsung is re...
...,...,...,...,...,...,...,...
295,"{'id': None, 'name': 'Core77.com'}",Rain Noe,Fantastic Industrial Design: The Kodak Smile C...,When Kodak wanted to produce an instant digita...,https://www.core77.com/posts/136031/Fantastic-...,2025-03-24T16:00:00Z,When Kodak wanted to produce an instant digita...
296,"{'id': 'business-insider', 'name': 'Business I...",jmcgrath@insider.com (Jenny McGrath),A sheet's thread count isn't as important as y...,"When it comes to sheets, you'll want to consid...",https://www.businessinsider.com/guides/home/wh...,2025-03-24T19:27:52Z,Thread count referring to the number of yarns ...
297,"{'id': None, 'name': 'Lifeinsuranceinternation...",GlobalData,1st Central adopts Clearspeed voice-based tech...,This move aims to expedite the settlement of l...,https://www.lifeinsuranceinternational.com/new...,2025-03-25T16:45:07Z,"1st Central, a UK-based insurer, has integrate..."
298,"{'id': None, 'name': 'Substack.com'}",UNTOPIA,The Tranhumanist Cult Test,Transhumanism is put through the 8-point test ...,https://ewanmorrison.substack.com/p/the-tranhu...,2025-03-24T12:06:16Z,In trying to have a few conversations about tr...


In [14]:
final_df=combined_articles_df.drop_duplicates(subset=["description"],keep='first')

In [15]:
final_df['publishedAt'] = pd.to_datetime(final_df['publishedAt']).dt.strftime('%Y-%m-%d')


/var/folders/_k/y3vww_pn5p14wn86g8x4k5mh0000gn/T/ipykernel_87955/1590193823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['publishedAt'] = pd.to_datetime(final_df['publishedAt']).dt.strftime('%Y-%m-%d')


In [16]:
# Method 1: Safe approach with type checking
import ast

def extract_source_name(source):
    if isinstance(source, dict):
        return source.get('name', 'Unknown')
    elif isinstance(source, str):
        try:
            dict_data = ast.literal_eval(source)  # Attempt to parse it as a dictionary
            if isinstance(dict_data, dict):
                return dict_data.get('name', 'Unknown')
        except (SyntaxError, ValueError):
            # If source is a plain string, return it directly
            return source
    return 'Unknown'


final_df.loc[:, 'source'] = final_df['source'].apply(extract_source_name)


In [17]:
len(final_df)

276